<a href="https://colab.research.google.com/github/abhranil-datascience/ArtificialNeuralNetworkPOC/blob/master/ClassificationANNPOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
############################## Mount Drive ######################################## 
from google.colab import drive
drive.mount('/content/gdrive')
############################## Change Directory ###################################
import os
os.chdir('/content/gdrive/My Drive/MLandDLFullCourse/DL/ANN/Classification')
########## Suppress Warnings ############
import warnings
warnings.filterwarnings('ignore')
########## Import Dataset ############
import pandas as pd
dataset=pd.read_csv('Churn_Modelling.csv')
X=dataset.iloc[:,3:13].values
Y=dataset.iloc[:,13].values
########## Preprocess Dataset #############
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,StandardScaler
le_Geography=LabelEncoder()
X[:,1]=le_Geography.fit_transform(X[:,1])
le_Gender=LabelEncoder()
X[:,2]=le_Gender.fit_transform(X[:,2])
oe=OneHotEncoder(categorical_features=[1])
X=oe.fit_transform(X).toarray()
X=X[:,1:]
sc=StandardScaler()
XScaled=sc.fit_transform(X)
########## Train Test Split ###############
from sklearn.model_selection import train_test_split
XTrain,XTest,YTrain,YTest=train_test_split(XScaled,Y,test_size=0.1,random_state=42)
######### Check VIF #################
from statsmodels.stats.outliers_influence import variance_inflation_factor
VIF=[0]*XTrain.shape[1]
for col in range(0,XTrain.shape[1]):
  VIF[col]=VIF[col]+variance_inflation_factor(exog=XTrain,exog_idx=col)
VIF

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


[1.3435397888131195,
 1.1251919091748992,
 1.001288233416671,
 1.0031336341046257,
 1.012815770532728,
 1.0022273669460904,
 1.3379193866256232,
 1.1186968996643618,
 1.002143963851318,
 1.0111334659604707,
 1.00119909969637]

In [0]:
######### Create ANN ###################
filepath="bestregressionmodel.h5"
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint
save_best_model=ModelCheckpoint(filepath,monitor='acc',save_best_only=True,verbose=1,mode='max')
ClassifierANN=Sequential()
ClassifierANN.add(Dense(input_shape=(11,),units=1024,activation='relu',kernel_initializer='random_uniform'))
ClassifierANN.add(Dropout(0.2))
ClassifierANN.add(Dense(units=2048,activation='relu',kernel_initializer='random_uniform'))
ClassifierANN.add(Dropout(0.2))
ClassifierANN.add(Dense(units=1024,activation='relu',kernel_initializer='random_uniform'))
ClassifierANN.add(Dropout(0.2))
ClassifierANN.add(Dense(units=1,activation='sigmoid',kernel_initializer='random_uniform'))
ClassifierANN.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
ClassifierANN.fit(x=XTrain,y=YTrain,batch_size=5,epochs=100,callbacks=[save_best_model])




Epoch 1/100





9000/9000 [==============================] - 13s 1ms/step - loss: 0.4116 - acc: 0.8337

Epoch 00001: acc improved from -inf to 0.83367, saving model to bestregressionmodel.h5
Epoch 2/100
9000/9000 [==============================] - 9s 980us/step - loss: 0.3818 - acc: 0.8518

Epoch 00002: acc improved from 0.83367 to 0.85178, saving model to bestregressionmodel.h5
Epoch 3/100
9000/9000 [==============================] - 9s 953us/step - loss: 0.3669 - acc: 0.8556

Epoch 00003: acc improved from 0.85178 to 0.85556, saving model to bestregressionmodel.h5
Epoch 4/100
9000/9000 [==============================] - 9s 967us/step - loss: 0.3622 - acc: 0.8571

Epoch 00004: acc improved from 0.85556 to 0.85711, saving model to bestregressionmodel.h5
Epoch 5/100
9000/9000 [==============================] - 9s 960us/step - loss: 0.3623 - acc: 0.8576

Epoch 00005: acc improved from 0.85711 to 0.85756, saving model to bestregressionmodel.h5
Epoch 6/100
9000/9000 [==================

In [0]:
######## Model Prediction #################
best_model=load_model(filepath)
YPred=best_model.predict(XTest)
YPred[YPred>=0.5]=1
YPred[YPred<0.5]=0
from sklearn import metrics as mt
print("Accuracy: "+str(mt.accuracy_score(YTest,YPred)))
cm=mt.confusion_matrix(YTest,YPred)
cm

Accuracy: 0.853


array([[771,  38],
       [109,  82]])